# Image Offsets: Create offset arrays used by drizzle

The offset tables created by a previous notebook (Offsets_2) are used to generate the X and Y offset arrays used by drizzle.

Each offset table will generate 2 arrays, for X and Y respectively, stored as FITS image of short float type.

The algorithms were developed in the Timing notebook; here, they are cast as callable functions used in a loop to process all images in the sequence.

In [1]:
import os, glob
import numpy as np

from astropy.table import Table

import rawpy

In [2]:
datadir = '../astrophotography_data/MilkyWayPrettyBoy/12800/light/'

## Functions

In [3]:
# Comparison functions
gt_zero = lambda x: x > 0.0
lt_zero = lambda x: x < 0.0

# Gets the index of the closest star in table. 
# The differences are in the sense pixel - star centroid.
# The comparison functions define from which quadrant the star is drawn from.
def closest(diff_x, diff_y, compare_x, compare_y):
    # Compute mask that masks out everything that is outside 
    # the quadrant defined by the comparison functions
    mask_x = np.where(compare_x(diff_x), 1, 0)
    mask_y = np.where(compare_y(diff_y), 1, 0)
    mask = mask_x * mask_y

    # Get index of star at minimum distance
    distance = np.sqrt((diff_x * diff_x + diff_y * diff_y)) * mask
    if np.nonzero(distance)[0].size > 0:
        mindist = np.min(distance[np.nonzero(distance)])
        index = np.where(distance == mindist )[0][0]
        return index, mindist
    else:
        return -1, 0.0

## Read last table in sequence, and prototype image

Starting with the last table ensures that we get always the same stars along the entire sequence. Offset tables at the beginning of the sequence may include stars that are dropped later on.

In [4]:
# last table in sequence
table_list = glob.glob(datadir + '/*.offsets_table.fits')
table_list.sort()
last = table_list[-1]
offsets_table = Table.read(last)

In [5]:
# prototype image (used to define array size)
image_name = last.split('/')[-1]
image_name = image_name.replace('.offsets_table.fits', '.ARW')
image_name = os.path.join(datadir, image_name)
raw = rawpy.imread(image_name)
imarray = raw.raw_image_visible.astype(float)

ny = imarray.shape[0]
nx = imarray.shape[1]

print(nx, ny)

4256 2848


## Create and populate arrays with interpolated offsets

In [6]:
offset_array_x = np.asarray(imarray) * 0.0
offset_array_y = np.asarray(imarray) * 0.0

In [7]:
centroid_x = offsets_table['xcentroid'].data
centroid_y = offsets_table['ycentroid'].data
offset_x = offsets_table['xoffset'].data
offset_y = offsets_table['yoffset'].data

In [8]:
for i in range(0, nx, 30):
    for j in range(0, ny, 30):
# for i in range(2000, 2001):
#     for j in range(1000, 1003):

        pixel_x = i
        pixel_y = j

        diff_x = pixel_x - centroid_x
        diff_y = pixel_y - centroid_y
        
        index = np.array(range(4), dtype=int)
        dist  = np.array(range(4), dtype=float)
        
        # get index and distance of the closest star, one per quadrant
        index[0], dist[0] = closest(diff_x, diff_y, gt_zero, gt_zero)
        index[1], dist[1] = closest(diff_x, diff_y, lt_zero, gt_zero)
        index[2], dist[2] = closest(diff_x, diff_y, gt_zero, lt_zero)
        index[3], dist[3] = closest(diff_x, diff_y, lt_zero, lt_zero)
        
        # need a cleanup here. Negative indices, zeroed distances.

        # weighted average of the offset values. The weight is the inverse distance pixel-star.
        sumweights = 1./dist[0] + 1./dist[1] + 1./dist[2] + 1./dist[3]
        weighted_offset_x = offset_x[index[0]] * (1./dist[0] / sumweights) + \
                            offset_x[index[1]] * (1./dist[1] / sumweights) + \
                            offset_x[index[2]] * (1./dist[2] / sumweights) + \
                            offset_x[index[3]] * (1./dist[3] / sumweights)
        weighted_offset_y = offset_y[index[0]] * (1./dist[0] / sumweights) + \
                            offset_y[index[1]] * (1./dist[1] / sumweights) + \
                            offset_y[index[2]] * (1./dist[2] / sumweights) + \
                            offset_y[index[3]] * (1./dist[3] / sumweights)

        offset_array_x[j][i] = weighted_offset_x
        offset_array_y[j][i] = weighted_offset_y



        



/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in double_sc

In [13]:
offset_array_x[1940:1960,1940:1960]

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,  